# 11 - Counterfactual Calculation

In [362]:
import pandas as pd
import pickle

In [363]:
# Display all rows and columns
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns

In [364]:
df = pd.read_parquet("../data/processed/X_test.parquet")

In [365]:
df.head()

,Position_IT Support,Position_Production Technician I,Position_Area Sales Manager,Position_Production Manager,Position_Production Technician II,Position_Sales Manager,Position_Enterprise Architect,Position_Network Engineer,Position_Sr. Network Engineer,Position_Database Administrator,Position_Data Analyst,Position_Software Engineer,Position_Sr. DBA,Position_Sr. Accountant,Position_Administrative Assistant,Position_Accountant I,Position_Shared Services Manager,Position_IT Director,Position_CIO,Position_Principal Data Architect,Position_IT Manager - DB,Position_IT Manager - Support,Position_IT Manager - Infra,Position_BI Developer,Position_Senior BI Developer,Position_Data Architect,Position_BI Director,Position_Director of Sales,Position_Director of Operations,Position_Software Engineering Manager,Position_President & CEO,State,Sex,CitizenDesc_US Citizen,CitizenDesc_Eligible NonCitizen,CitizenDesc_Non-Citizen,HispanicLatino,RaceDesc_White,RaceDesc_Black or African American,RaceDesc_Asian,RaceDesc_American Indian or Alaska Native,RaceDesc_Hispanic,RaceDesc_Two or more races,Department_IT/IS,Department_Production,Department_Sales,Department_Software Engineering,Department_Admin Offices,Department_Executive Office,YearsExperience,AgeGroup,ExperienceCategory,Education,Advanced Backup Strategies,Advanced Budget Forecasting,Advanced CRM Tools,Advanced Data Modeling,Advanced Data Visualization,Advanced Financial Reporting,Advanced Firewall Configurations,Advanced ITSM Tools,Advanced Machinery Maintenance,Advanced Machinery Troubleshooting,Advanced Network Configuration,Advanced Predictive Modeling,Advanced Revenue Analysis,Advanced SQL Optimization,Advanced Troubleshooting Techniques,Advanced Visualization,Agile Development Leadership,Audit Assistance,Audit Management,Backup Strategies,Backup and Recovery,Basic Accounting,Basic Machinery Maintenance,Big Data Architecture,Big Data Solutions,Budget Oversight,Budget Planning,Budget Strategy,Business Intelligence Strategy,Business Intelligence Tools,Business-IT Alignment,CI/CD Pipeline Management,Cloud Data Management,Cloud Data Solutions,Cloud Database Solutions,Cloud Integration,Cloud Networking,Cloud Strategy,Cloud-Native Data Architectures,Code Review Practices,Competitor Analysis,Cost Reduction Techniques,Customer Communication,Customer Relationship Management,Customer Retention,Customer Support,Customer Support Strategies,Cybersecurity Oversight,Dashboard Creation,Data Governance,Data Lake Architecture,Data Modeling,Data Pipeline Optimization,Data Pipeline Scalability,Data Security,Data Visualization,Database Design,Database Management,Database Tuning,Disaster Recovery Planning,Distributed Database Management,Document Management,ETL Automation,ETL Development,ETL Optimization,Efficiency Optimization,Enterprise Data Strategy,Financial Management,Financial Reporting,Firewall Expertise,Firewall Management,Forensic Accounting Techniques,Governance and Standards,Hardware Maintenance,Hardware Management,Hybrid Cloud Infrastructure Management,IT Governance,IT Security Oversight,IT Support Management,Incident Response Planning,Infrastructure Design,Java,Leadership,Leadership Skills,Lean Manufacturing,Machine Learning,Machine Learning Integration,Market Analysis,Microservices Architecture Design,Negotiation,Network Configuration,Network Management,Network Performance Optimization,Network Security Design,Office Coordination,Operations Performance Metrics,Operations Strategy,Performance Tuning,Predictive Analytics Integration,Preventive Maintenance Planning,Problem Identification,Problem-Solving,Process Improvement,Process Optimization,Production Line Efficiency Analysis,Public Relations,Python,Quality Assurance,QuickBooks,Real-Time Data Processing,Revenue Optimization,Risk Assessment,SD-WAN Deployment,SQL,SQL Optimization,Safety Protocols,Sales Funnel Optimization,Sales Strategy,Scheduling,Service Delivery Optimization,Software Design,Solution Architecture,Statistical Analysis,Strategic IT In

In [366]:
# Load the model
model_path = "../models/xgb_model.pkl"
with open(model_path, 'rb') as file:
    xgb_model = pickle.load(file)

In [367]:
probs = xgb_model.predict_proba(df)[:, 1]  # Get the probability of the positive class

In [368]:
# Append probabilities as a new column in the dataframe
df['probability'] = probs

In [369]:
# Define thresholds for filtering
threshold_lower = 0.45
threshold_upper = 0.55

In [370]:
# Filter rows based on probabilities being close to the threshold range (e.g., 0.45 to 0.5)
filtered_df = df[(df['probability'] >= threshold_lower) & (df['probability'] <= threshold_upper)]

In [371]:
df.YearsExperience.value_counts()

YearsExperience
13    77
10    70
11    51
12    40
14    24
9     18
8     14
16     8
7      4
15     3
17     3
19     1
Name: count, dtype: int64

In [372]:
filtered_df.head()

,Position_IT Support,Position_Production Technician I,Position_Area Sales Manager,Position_Production Manager,Position_Production Technician II,Position_Sales Manager,Position_Enterprise Architect,Position_Network Engineer,Position_Sr. Network Engineer,Position_Database Administrator,Position_Data Analyst,Position_Software Engineer,Position_Sr. DBA,Position_Sr. Accountant,Position_Administrative Assistant,Position_Accountant I,Position_Shared Services Manager,Position_IT Director,Position_CIO,Position_Principal Data Architect,Position_IT Manager - DB,Position_IT Manager - Support,Position_IT Manager - Infra,Position_BI Developer,Position_Senior BI Developer,Position_Data Architect,Position_BI Director,Position_Director of Sales,Position_Director of Operations,Position_Software Engineering Manager,Position_President & CEO,State,Sex,CitizenDesc_US Citizen,CitizenDesc_Eligible NonCitizen,CitizenDesc_Non-Citizen,HispanicLatino,RaceDesc_White,RaceDesc_Black or African American,RaceDesc_Asian,RaceDesc_American Indian or Alaska Native,RaceDesc_Hispanic,RaceDesc_Two or more races,Department_IT/IS,Department_Production,Department_Sales,Department_Software Engineering,Department_Admin Offices,Department_Executive Office,YearsExperience,AgeGroup,ExperienceCategory,Education,Advanced Backup Strategies,Advanced Budget Forecasting,Advanced CRM Tools,Advanced Data Modeling,Advanced Data Visualization,Advanced Financial Reporting,Advanced Firewall Configurations,Advanced ITSM Tools,Advanced Machinery Maintenance,Advanced Machinery Troubleshooting,Advanced Network Configuration,Advanced Predictive Modeling,Advanced Revenue Analysis,Advanced SQL Optimization,Advanced Troubleshooting Techniques,Advanced Visualization,Agile Development Leadership,Audit Assistance,Audit Management,Backup Strategies,Backup and Recovery,Basic Accounting,Basic Machinery Maintenance,Big Data Architecture,Big Data Solutions,Budget Oversight,Budget Planning,Budget Strategy,Business Intelligence Strategy,Business Intelligence Tools,Business-IT Alignment,CI/CD Pipeline Management,Cloud Data Management,Cloud Data Solutions,Cloud Database Solutions,Cloud Integration,Cloud Networking,Cloud Strategy,Cloud-Native Data Architectures,Code Review Practices,Competitor Analysis,Cost Reduction Techniques,Customer Communication,Customer Relationship Management,Customer Retention,Customer Support,Customer Support Strategies,Cybersecurity Oversight,Dashboard Creation,Data Governance,Data Lake Architecture,Data Modeling,Data Pipeline Optimization,Data Pipeline Scalability,Data Security,Data Visualization,Database Design,Database Management,Database Tuning,Disaster Recovery Planning,Distributed Database Management,Document Management,ETL Automation,ETL Development,ETL Optimization,Efficiency Optimization,Enterprise Data Strategy,Financial Management,Financial Reporting,Firewall Expertise,Firewall Management,Forensic Accounting Techniques,Governance and Standards,Hardware Maintenance,Hardware Management,Hybrid Cloud Infrastructure Management,IT Governance,IT Security Oversight,IT Support Management,Incident Response Planning,Infrastructure Design,Java,Leadership,Leadership Skills,Lean Manufacturing,Machine Learning,Machine Learning Integration,Market Analysis,Microservices Architecture Design,Negotiation,Network Configuration,Network Management,Network Performance Optimization,Network Security Design,Office Coordination,Operations Performance Metrics,Operations Strategy,Performance Tuning,Predictive Analytics Integration,Preventive Maintenance Planning,Problem Identification,Problem-Solving,Process Improvement,Process Optimization,Production Line Efficiency Analysis,Public Relations,Python,Quality Assurance,QuickBooks,Real-Time Data Processing,Revenue Optimization,Risk Assessment,SD-WAN Deployment,SQL,SQL Optimization,Safety Protocols,Sales Funnel Optimization,Sales Strategy,Scheduling,Service Delivery Optimization,Software Design,Solution Architecture,Statistical Analysis,Strategic IT In

In [373]:
row_to_modify = filtered_df.iloc[1].copy()

In [374]:
# Display the original row and its probability
print("Original Row:")
print(row_to_modify)

Original Row:
Position_IT Support                                             0.0000
Position_Production Technician I                                1.0000
Position_Area Sales Manager                                     0.0000
Position_Production Manager                                     0.0000
Position_Production Technician II                               0.0000
Position_Sales Manager                                          0.0000
Position_Enterprise Architect                                   0.0000
Position_Network Engineer                                       0.0000
Position_Sr. Network Engineer                                   0.0000
Position_Database Administrator                                 0.0000
Position_Data Analyst                                           0.0000
Position_Software Engineer                                      0.0000
Position_Sr. DBA                                                0.0000
Position_Sr. Accountant                                        

In [375]:
# Modify the row: change RaceDesc_White from 1 to 0 and RaceDesc_Asian from 0 to 1
if 'RaceDesc_White' in row_to_modify.index and 'RaceDesc_Asian' in row_to_modify.index:
    row_to_modify['RaceDesc_White'] = 0
    row_to_modify['RaceDesc_Asian'] = 1

In [376]:
# Display the modified row
print("\nModified Row:")
print(row_to_modify)


Modified Row:
Position_IT Support                                             0.0000
Position_Production Technician I                                1.0000
Position_Area Sales Manager                                     0.0000
Position_Production Manager                                     0.0000
Position_Production Technician II                               0.0000
Position_Sales Manager                                          0.0000
Position_Enterprise Architect                                   0.0000
Position_Network Engineer                                       0.0000
Position_Sr. Network Engineer                                   0.0000
Position_Database Administrator                                 0.0000
Position_Data Analyst                                           0.0000
Position_Software Engineer                                      0.0000
Position_Sr. DBA                                                0.0000
Position_Sr. Accountant                                       

In [377]:
# Convert the row back into a DataFrame (model expects a DataFrame for predictions)
modified_row_df = row_to_modify.to_frame().T
probability = modified_row_df.probability
probability

663    0.4816
Name: probability, dtype: float64

In [378]:
modified_row_df.drop(columns="probability", inplace=True)

In [379]:
# Recalculate the probability for the modified row
counterfactual_probability = xgb_model.predict_proba(modified_row_df)[:, 1][0]

# Display the counterfactual probability
print("\nCounterfactual Probability:")
print(counterfactual_probability)


Counterfactual Probability:
0.9940832


The counterfactual probability of **0.99927** shows that changing `RaceDesc_White` to 0 and `RaceDesc_Asian` to 1 caused the model to predict the positive class with almost 100% confidence. This suggests the model might be overly sensitive to these features, possibly due to their high importance or correlations in the training data.

## **Modifiable Attributes : gender, age, race**

In [380]:
def modify_and_recalculate_probability(row: pd.Series, 
                                      xgb_model: object, 
                                      attribute: str, 
                                      new_value: int | str) -> dict:
    """
    Modify a single attribute of a selected row, recalculate probability, and return results.

    Parameters:
    row (pd.Series): The original row selected by the user.
    xgb_model: The trained XGBoost model for predicting probabilities.
    attribute (str): The attribute to modify ('Sex', 'Race', or 'YearsExperience').
    new_value: The new value to set for the selected attribute.
        - For 'Sex': 0 (Female) or 1 (Male)
        - For 'Race': 'White', 'Black', or 'Asian'
        - For 'YearsExperience': 7, 13, or 17

    Returns:
    dict: A dictionary containing the original probability, modified row, and counterfactual probability.
    """
    import pandas as pd

    # Validate inputs
    if attribute not in ['Sex', 'Race', 'YearsExperience']:
        raise ValueError("Invalid attribute. Must be 'Sex', 'Race', or 'YearsExperience'.")

    if attribute == 'Sex' and new_value not in [0, 1]:
        raise ValueError("Invalid value for 'Sex'. Must be 0 (Female) or 1 (Male).")

    if attribute == 'Race' and new_value not in ['White', 'Black', 'Asian']:
        raise ValueError("Invalid value for 'Race'. Must be 'White', 'Black', or 'Asian")

    if attribute == 'YearsExperience' and new_value not in [7, 13, 17]:
        raise ValueError("Invalid value for 'YearsExperience'. Must be 7, 13, or 17.")

    # Copy the row to avoid modifying the original data
    modified_row = row.copy()

    # Save the original probability
    original_probability = modified_row.get('probability', None)
    if original_probability is None:
        raise KeyError("The column 'probability' is missing in the row.")

    # Modify the specified attribute
    if attribute == 'Sex':
        modified_row['Sex'] = new_value
    elif attribute == 'Race':
        # Dynamically identify all columns that start with 'RaceDesc_'
        race_columns = [col for col in modified_row.index if col.startswith('RaceDesc_')]
        # Reset all race columns to 0
        for col in race_columns:
            modified_row[col] = 0

        # Set the selected race column to 1
        race_column_mapping = {
            'White': 'RaceDesc_White',
            'Black': 'RaceDesc_Black or African American',
            'Asian': 'RaceDesc_Asian'
        }
        race_col = race_column_mapping.get(new_value)
        if race_col and race_col in modified_row:
            modified_row[race_col] = 1
    elif attribute == 'YearsExperience':
        modified_row['YearsExperience'] = new_value

    # Convert the modified row to a DataFrame (XGBoost model expects this format)
    modified_row_df = modified_row.to_frame().T
    modified_row_df.drop(columns="probability", inplace=True)

    # Recalculate probability for the modified row
    counterfactual_probability = xgb_model.predict_proba(modified_row_df)[:, 1][0]

    return {
        "original_probability": original_probability,
        "counterfactual_probability": counterfactual_probability,
        "modified_row": modified_row
    }

In [381]:
result = modify_and_recalculate_probability(row_to_modify, xgb_model, 'Race', 'Asian')
print(result)

{'original_probability': np.float64(0.4816003441810608), 'counterfactual_probability': np.float32(0.9940832), 'modified_row': Position_IT Support                                             0.0000
Position_Production Technician I                                1.0000
Position_Area Sales Manager                                     0.0000
Position_Production Manager                                     0.0000
Position_Production Technician II                               0.0000
Position_Sales Manager                                          0.0000
Position_Enterprise Architect                                   0.0000
Position_Network Engineer                                       0.0000
Position_Sr. Network Engineer                                   0.0000
Position_Database Administrator                                 0.0000
Position_Data Analyst                                           0.0000
Position_Software Engineer                                      0.0000
Position_Sr. DBA      

In [382]:
# Filter for a row where 'RaceDesc_White' is 1
white_row = df[df['RaceDesc_White'] == 1].iloc[0]

# Change race from 'White' to 'Black'
result = modify_and_recalculate_probability(white_row, xgb_model, 'Race', 'Black')

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])

Original Probability: 0.5309375524520874
Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                0.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               1.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000

In [383]:
# Filter for a row where 'Sex' is 1 (Male)
male_row = filtered_df[filtered_df['Sex'] == 1].iloc[0]

# Change gender from Male to Female
result = modify_and_recalculate_probability(male_row, xgb_model, 'Sex', 0)

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])

Original Probability: 0.4816003441810608
Modified Row:
Position_IT Support                                             0.0000
Position_Production Technician I                                1.0000
Position_Area Sales Manager                                     0.0000
Position_Production Manager                                     0.0000
Position_Production Technician II                               0.0000
Position_Sales Manager                                          0.0000
Position_Enterprise Architect                                   0.0000
Position_Network Engineer                                       0.0000
Position_Sr. Network Engineer                                   0.0000
Position_Database Administrator                                 0.0000
Position_Data Analyst                                           0.0000
Position_Software Engineer                                      0.0000
Position_Sr. DBA                                                0.0000
Position_Sr. Accountan

In [384]:
# Filter for a row with a specific value for 'YearsExperience' (e.g., 7, 13, or 17)
years_row = df[df['YearsExperience'] == 7].iloc[1]

# Change 'YearsExperience' to a new value (e.g., 13)
result = modify_and_recalculate_probability(years_row, xgb_model, 'YearsExperience', 17)

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])


Original Probability: 0.7197088003158569
Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                0.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000

In [385]:
# Filter for a row with a specific value for 'YearsExperience' (e.g., 7, 13, or 17)
years_row = filtered_df[filtered_df['YearsExperience'] == 13].iloc[0]

# Change 'YearsExperience' to a new value (e.g., 17)
result = modify_and_recalculate_probability(years_row, xgb_model, 'YearsExperience', 17)

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])

Original Probability: 0.4788358211517334
Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                1.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000

By changing individual attributes like race, gender, or years of experience, we observed significant shifts in the model's predictions. These results highlight the sensitivity of the model to specific features and emphasize the need to evaluate feature importance and potential biases. This demonstrates the impact of single-value modifications on prediction outcomes.

In [386]:
# Classification threshold (e.g., 0.5)
classification_threshold = 0.5

candidates = []

# Define attributes and values to modify
attributes = [
    {"attribute": "Sex", "values": [0, 1]},  # Female to Male or vice versa
    {"attribute": "Race", "values": ["White", "Black", "Asian"]},  # Race changes (mapped to columns)
    {"attribute": "YearsExperience", "values": [7, 13, 17]}  # Experience levels
]

# Mapping for race columns (one-hot encoded)
race_column_mapping = {
    "White": "RaceDesc_White",
    "Black": "RaceDesc_Black or African American",
    "Asian": "RaceDesc_Asian"
}

# Mapping for ExperienceCategory based on YearsExperience
experience_category_mapping = {
    7: 0,   # 6-10 years -> ExperienceCategory = 0
    13: 1,  # 11-20 years -> ExperienceCategory = 1
    17: 1   # 11-20 years -> ExperienceCategory = 1
}

# Iterate through rows to identify significant changes
for idx, row in df.iterrows():
    for attr in attributes:
        if attr["attribute"] == "Race":
            # Handle one-hot encoding for race
            for race, race_col in race_column_mapping.items():
                if race_col in row and row[race_col] == 1:  # Skip if current race matches target race
                    continue

                # Create a modified copy of the row for race modification
                modified_row = row.copy()

                # Reset all race columns to 0
                for col in race_column_mapping.values():
                    if col in modified_row:
                        modified_row[col] = 0

                # Set the target race column to 1
                if race_col in modified_row:
                    modified_row[race_col] = 1

                # Apply modification and calculate probabilities
                result = modify_and_recalculate_probability(modified_row, xgb_model, "Race", race)
                original_prob = result["original_probability"]
                counterfactual_prob = result["counterfactual_probability"]

                # Identify the original race
                original_race = next(
                    (k for k, v in race_column_mapping.items() if v in row and row[v] == 1),
                    "Unknown"
                )

                # Check if the change crosses the classification threshold
                if (original_prob < classification_threshold <= counterfactual_prob) or \
                   (original_prob >= classification_threshold > counterfactual_prob):
                    candidates.append({
                        "row_index": idx,
                        "attribute": "Race",
                        "original_value": original_race,
                        "new_value": race,
                        "original_probability": original_prob,
                        "counterfactual_probability": counterfactual_prob,
                        "probability_change": abs(counterfactual_prob - original_prob)
                    })
        elif attr["attribute"] == "YearsExperience":
            # Handle YearsExperience changes
            for value in attr["values"]:
                if row[attr["attribute"]] == value:  # Skip if the current value matches the new value
                    continue

                # Create a modified copy of the row
                modified_row = row.copy()

                # Set the new value for YearsExperience
                modified_row[attr["attribute"]] = value

                # Set the corresponding ExperienceCategory based on the mapping
                if value in experience_category_mapping:
                    modified_row["ExperienceCategory"] = experience_category_mapping[value]

                # Apply modification and calculate probabilities
                result = modify_and_recalculate_probability(modified_row, xgb_model, attr["attribute"], value)
                original_prob = result["original_probability"]
                counterfactual_prob = result["counterfactual_probability"]

                # Check if the change crosses the classification threshold
                if (original_prob < classification_threshold <= counterfactual_prob) or \
                   (original_prob >= classification_threshold > counterfactual_prob):
                    candidates.append({
                        "row_index": idx,
                        "attribute": attr["attribute"],
                        "original_value": row[attr["attribute"]],
                        "new_value": value,
                        "original_probability": original_prob,
                        "counterfactual_probability": counterfactual_prob,
                        "probability_change": abs(counterfactual_prob - original_prob),
                        "adjusted_ExperienceCategory": experience_category_mapping[value]  # Log updated category
                    })
        else:
            # Handle other attributes (e.g., Sex)
            for value in attr["values"]:
                if row[attr["attribute"]] == value:  # Skip if the current value matches the new value
                    continue

                # Apply modification and calculate probabilities
                result = modify_and_recalculate_probability(row, xgb_model, attr["attribute"], value)
                original_prob = result["original_probability"]
                counterfactual_prob = result["counterfactual_probability"]

                # Check if the change crosses the classification threshold
                if (original_prob < classification_threshold <= counterfactual_prob) or \
                   (original_prob >= classification_threshold > counterfactual_prob):
                    candidates.append({
                        "row_index": idx,
                        "attribute": attr["attribute"],
                        "original_value": row[attr["attribute"]],
                        "new_value": value,
                        "original_probability": original_prob,
                        "counterfactual_probability": counterfactual_prob,
                        "probability_change": abs(counterfactual_prob - original_prob)
                    })

In [387]:
# Sort by probability change for better insights
candidates_df = pd.DataFrame(candidates)
candidates_df = candidates_df.sort_values(by="probability_change", ascending=False)
candidates_df = candidates_df.reset_index(drop=True)

In [388]:
# Define custom sort order for the attribute column
attribute_order = {"Sex": 0, "Race": 1, "YearsExperience": 2}

# Sort DataFrame
candidates_df = candidates_df.sort_values(
    by=["attribute", "original_probability", "probability_change"],
    key=lambda col: col.map(attribute_order) if col.name == "attribute" else col,
    ascending=[True, True, False]  # Ascending for attribute and original_probability, descending for probability_change
)
candidates_df.reset_index(drop=True, inplace=True)
candidates_df

,row_index,attribute,original_value,new_value,original_probability,counterfactual_probability,probability_change,adjusted_ExperienceCategory
0,1024,Sex,0.0,1,0.013869,0.987069,0.973200,NaN
1,186,Sex,0.0,1,0.049773,0.996460,0.946687,NaN
2,1314,Sex,0.0,1,0.066717,0.725554,0.658838,NaN
3,1023,Sex,0.0,1,0.084336,0.740908,0.656572,NaN
4,916,Sex,0.0,1,0.118637,0.950724,0.832087,NaN
5,2437,Sex,1.0,0,0.121169,0.993853,0.872684,NaN
6,1328,Sex,0.0,1,0.122297,0.687214,0.564918,NaN
7,1972,Sex,0.0,1,0.129886,0.994001,0.864115,NaN
8,1939,Sex,0.0,1,0.134728,0.998122,0.863393,NaN
9,2302,Sex,0.0,1,0.170168,0.883811,0.713643,NaN


In [389]:
# Group by 'row_index' and count occurrences
duplicate_candidates = candidates_df.groupby("row_index").size()

# Filter for row indices that appear more than once
duplicate_row_indices = duplicate_candidates[duplicate_candidates > 1].index

# Filter original DataFrame for these row indices and select required columns
duplicates_df = candidates_df[candidates_df["row_index"].isin(duplicate_row_indices)][
    ["row_index", "attribute", "original_value", "new_value", "original_probability", "counterfactual_probability", "probability_change"]
]
duplicates_df.reset_index(drop=True, inplace=True)
duplicates_df.sort_values(by="row_index")

,row_index,attribute,original_value,new_value,original_probability,counterfactual_probability,probability_change
26,13,Sex,1.0,0,0.885757,0.091377,0.794381
208,13,YearsExperience,13.0,7,0.885757,0.014956,0.870802
99,49,Race,White,Asian,0.572846,0.057260,0.515586
98,49,Race,White,Black,0.572846,0.021262,0.551584
61,62,Race,White,Black,0.216507,0.999293,0.782786
157,62,YearsExperience,10.0,17,0.216507,0.516125,0.299618
64,62,Race,White,Asian,0.216507,0.983369,0.766862
152,62,YearsExperience,10.0,7,0.216507,0.995906,0.779399
105,107,Race,White,Black,0.897358,0.010265,0.887093
210,107,YearsExperience,11.0,7,0.897358,0.373111,0.524247


**Note:** We can use this table to identify some interesting candidates for the MVP, as it highlights rows where multiple attributes significantly impact the classification threshold.

Examples: 1550, 1972, 2302

In [398]:
row_indices = [1550, 1972, 2302]  # Indices to select
selected_rows = df.loc[row_indices]

In [399]:
selected_rows.drop(columns="probability", inplace=True)

In [400]:
hr_data = pd.read_parquet("../data/interim/hr_data_simulated.parquet")
hr_selected_rows = hr_data.loc[row_indices]

In [401]:
hr_selected_rows["Employee_Name"]

1550    Rose, Abbigail
1972       Black, Kane
2302     Joyce, Kaylyn
Name: Employee_Name, dtype: object

In [402]:
# Drop 'probability' from selected_rows if it exists
if 'probability' in selected_rows.columns:
    selected_rows.drop(columns='probability', inplace=True)

# Select rows by indices in HR data
hr_selected_rows = hr_data.loc[row_indices]
hr_selected_rows = hr_selected_rows["Employee_Name"]

# Concatenate along columns (axis 1)
combined_rows = pd.concat([selected_rows, hr_selected_rows], axis=1)

In [403]:
hr_selected_rows

1550    Rose, Abbigail
1972       Black, Kane
2302     Joyce, Kaylyn
Name: Employee_Name, dtype: object

In [404]:
# Save the selected rows to a new parquet file without index
combined_rows.to_parquet("../data/app/static_data.parquet", index=False)